In [2]:
import pandas as pd
import numpy as np
import json
import datetime
from dateutil.parser import parse

In [11]:
with open("./train.json", 'r') as j:
     train = json.loads(j.read())

In [12]:
custom_columns=["tweet_id","timestamp","year","month","week_day","hour","is_reply_to_tweet","is_reply_to_user",
            "user_id","@_count","#_count","url_count","tweet_len", "rewteet_count", "favorite_count"]
aggregate_columns=["tweet_id","user_total_retweeted_so_far","user_total_favorited_so_far",
                   "user_total_retweeted", "user_total_favorited"]
#tweet id to count
features=[]
for i,twt in enumerate(train):
    row=[twt["id_str"]]
    time=parse(twt['created_at'])
    row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
    if twt["in_reply_to_status_id_str"]==None:
        row.append(0)
    else:
        row.append(1)

    if twt["in_reply_to_user_id_str"]==None:
        row.append(0)
    else:
        row.append(1)

    row.append(twt["user"]["id_str"])
    row.append(len(twt["entities"]["user_mentions"]))
    row.append(len(twt["entities"]["hashtags"]))
    row.append(len(twt["entities"]["urls"]))
    row.append(len(twt["text"]))
    row.append(twt["retweet_count"])
    row.append(twt["favorite_count"])

    features.append(row)

KeyError: 'retweet_count'

In [7]:
feature_df=pd.DataFrame(data=features, columns=custom_columns)
feature_df.sort_values(by=['timestamp'], inplace=True)

In [8]:
aggregate_features=[]
for user_id, group in feature_df.groupby('user_id'):
    #try:
    #    print(user_id,reply_to_user_dict[user_id])
    #except:
    #    print(user_id, 0)
    group_row=[]
    total_retweet=0
    total_favorite=0
    for twt in group.iterrows():
        row=[twt[1]["tweet_id"]]
        total_retweet+=twt[1]["rewteet_count"]
        total_favorite+=twt[1]["favorite_count"]
        row.extend([total_retweet, total_favorite])
        group_row.append(row)
    for i in range(len(group_row)):
        group_row[i].extend([total_retweet, total_favorite])
    aggregate_features.extend(group_row)

In [9]:
agg_df=pd.DataFrame(data=aggregate_features, columns=aggregate_columns)
custom_features_df=pd.merge(feature_df, agg_df, on="tweet_id")

In [10]:
custom_features_df.to_csv("./train_custom_features.csv",index=False)

In [13]:
with open("./test_questions.json", 'r') as j:
     test = json.loads(j.read())

In [16]:
custom_columns=["tweet_id","timestamp","year","month","week_day","hour","is_reply_to_tweet","is_reply_to_user",
            "user_id","@_count","#_count","url_count","tweet_len"]
#tweet id to count
features=[]
for i,twt in enumerate(train):
    row=[twt["id_str"]]
    time=parse(twt['created_at'])
    row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
    if twt["in_reply_to_status_id_str"]==None:
        row.append(0)
    else:
        row.append(1)

    if twt["in_reply_to_user_id_str"]==None:
        row.append(0)
    else:
        row.append(1)

    row.append(twt["user"]["id_str"])
    row.append(len(twt["entities"]["user_mentions"]))
    row.append(len(twt["entities"]["hashtags"]))
    row.append(len(twt["entities"]["urls"]))
    row.append(len(twt["text"]))

    features.append(row)

In [17]:
feature_df=pd.DataFrame(data=features, columns=custom_columns)
feature_df.sort_values(by=['timestamp'], inplace=True)